In [1]:
from datasets import load_dataset

dataset = load_dataset("qanastek/MASSIVE", "en-US", data_dir='MASSIVE')

Using custom data configuration en-US-data_dir=MASSIVE
Reusing dataset massive (C:\Users\andre\.cache\huggingface\datasets\qanastek___massive\en-US-data_dir=MASSIVE\1.0.0\31cdffab94ac97bfe5a394b1e96344c96f0ad847e1d796c7562d8c8b449e22e6)
100%|██████████| 3/3 [00:00<00:00, 39.47it/s]


In [2]:
_INTENTS = ['audio_volume_other', 'play_music', 'iot_hue_lighton', 'general_greet', 'calendar_set', 'audio_volume_down', 'social_query', 'audio_volume_mute', 'iot_wemo_on', 'iot_hue_lightup', 'audio_volume_up', 'iot_coffee', 'takeaway_query', 'qa_maths', 'play_game', 'cooking_query', 'iot_hue_lightdim', 'iot_wemo_off', 'music_settings', 'weather_query', 'news_query', 'alarm_remove', 'social_post', 'recommendation_events', 'transport_taxi', 'takeaway_order', 'music_query', 'calendar_query', 'lists_query', 'qa_currency', 'recommendation_movies', 'general_joke', 'recommendation_locations', 'email_querycontact', 'lists_remove', 'play_audiobook', 'email_addcontact', 'lists_createoradd', 'play_radio', 'qa_stock', 'alarm_query', 'email_sendemail', 'general_quirky', 'music_likeness', 'cooking_recipe', 'email_query', 'datetime_query', 'transport_traffic', 'play_podcasts', 'iot_hue_lightchange', 'calendar_remove', 'transport_query', 'transport_ticket', 'qa_factoid', 'iot_cleaning', 'alarm_set', 'datetime_convert', 'iot_hue_lightoff', 'qa_definition', 'music_dislikeness']
_TAGS = ['O', 'B-food_type', 'B-movie_type', 'B-person', 'B-change_amount', 'I-relation', 'I-game_name', 'B-date', 'B-movie_name', 'I-person', 'I-place_name', 'I-podcast_descriptor', 'I-audiobook_name', 'B-email_folder', 'B-coffee_type', 'B-app_name', 'I-time', 'I-coffee_type', 'B-transport_agency', 'B-podcast_descriptor', 'I-playlist_name', 'B-media_type', 'B-song_name', 'I-music_descriptor', 'I-song_name', 'B-event_name', 'I-timeofday', 'B-alarm_type', 'B-cooking_type', 'I-business_name', 'I-color_type', 'B-podcast_name', 'I-personal_info', 'B-weather_descriptor', 'I-list_name', 'B-transport_descriptor', 'I-game_type', 'I-date', 'B-place_name', 'B-color_type', 'B-game_name', 'I-artist_name', 'I-drink_type', 'B-business_name', 'B-timeofday', 'B-sport_type', 'I-player_setting', 'I-transport_agency', 'B-game_type', 'B-player_setting', 'I-music_album', 'I-event_name', 'I-general_frequency', 'I-podcast_name', 'I-cooking_type', 'I-radio_name', 'I-joke_type', 'I-meal_type', 'I-transport_type', 'B-joke_type', 'B-time', 'B-order_type', 'B-business_type', 'B-general_frequency', 'I-food_type', 'I-time_zone', 'B-currency_name', 'B-time_zone', 'B-ingredient', 'B-house_place', 'B-audiobook_name', 'I-ingredient', 'I-media_type', 'I-news_topic', 'B-music_genre', 'I-definition_word', 'B-list_name', 'B-playlist_name', 'B-email_address', 'I-currency_name', 'I-movie_name', 'I-device_type', 'I-weather_descriptor', 'B-audiobook_author', 'I-audiobook_author', 'I-app_name', 'I-order_type', 'I-transport_name', 'B-radio_name', 'I-business_type', 'B-definition_word', 'B-artist_name', 'I-movie_type', 'B-transport_name', 'I-email_folder', 'B-music_album', 'I-house_place', 'I-music_genre', 'B-drink_type', 'I-alarm_type', 'B-music_descriptor', 'B-news_topic', 'B-meal_type', 'I-transport_descriptor', 'I-email_address', 'I-change_amount', 'B-device_type', 'B-transport_type', 'B-relation', 'I-sport_type', 'B-personal_info']

In [3]:
def index_to_intent(index):
    return _INTENTS[index]

def index_to_tag(index):
    return _TAGS[index]

In [4]:
print(dataset['train'][0])

{'id': '1', 'locale': 'en-US', 'partition': 'train', 'scenario': 9, 'intent': 55, 'utt': 'wake me up at nine am on friday', 'annot_utt': 'wake me up at [time : nine am] on [date : friday]', 'tokens': ['wake', 'me', 'up', 'at', 'nine', 'am', 'on', 'friday'], 'ner_tags': [0, 0, 0, 0, 60, 16, 0, 7], 'worker_id': '1', 'slot_method': {'slot': [], 'method': []}, 'judgments': {'worker_id': [], 'intent_score': [], 'slots_score': [], 'grammar_score': [], 'spelling_score': [], 'language_identification': []}}


In [5]:
intent_slots = {}
for data in dataset['train']:
    intent = data['intent']
    if intent not in intent_slots:
        intent_slots[intent] = set(data['ner_tags'])
    else:
        intent_slots[intent] = intent_slots[intent].union(set(data['ner_tags']))

In [6]:
# print(intent_slots)

In [7]:
named_data = {}
for intent, slots in intent_slots.items():
    named_data[intent] = 'slots: ' + ", ".join(set([index_to_tag(slot)[2:] if index_to_tag(slot) != 'O' else 'O' for slot in slots]))

In [8]:
# print(named_data)

In [9]:
from torch.utils.data import Dataset
import torch

class MassiveDataset(Dataset):
    def __init__(self, data, class_info, tokenizer, max_len=128):
        self.data = data
        self.class_info = class_info
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        data  = self.data[index]
        intent = data['intent']
        intent_slots = self.class_info[intent]
        
        

    
    def __len__(self):
        return len(self.data)